In [3]:
import re
import pandas as pd
import numpy as np 
# import matplotlib.pyplot as plt
# import urllib.request
from collections import Counter
from konlpy.tag import Mecab
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from konlpy.tag import Okt
from tensorflow.keras.models import load_model

2023-06-06 11:04:37.658302: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-06 11:04:37.992662: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-06 11:04:37.994281: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-06 11:04:39.193448: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 10.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.2/464.2 kB 10.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 10.5 MB/s eta 0:00:0000:0100:01


In [3]:
# import tensorflow
# import keras
# import numpy as np
# import pandas as pd
# import h5py
# import konlpy 

# print('tensorflow ',tensorflow.__version__)
# print('keras', keras.__version__)
# print('numpy', np.__version__)
# print('padnas', pd.__version__)
# print('h5py', h5py.__version__)
# print('konlpy', konlpy.__version__)

tensorflow  2.12.0
keras 2.12.0
numpy 1.23.5
padnas 2.0.2
h5py 3.8.0
konlpy 0.6.0


In [8]:
class psng_predict_all:
    def __init__(self, df):
        self.df = df
        path = ''
        self.loaded_model = load_model(f'{path}new_model_01_v2.h5')
        train_data = pd.read_csv(f'{path}train_data_main.csv', encoding='euc-kr')
        test_data = pd.read_csv(f'{path}test_data_main.csv', encoding='euc-kr')
        X_train = train_data['tokenized'].apply(eval).values
        self.X_test = test_data['tokenized'].apply(eval).values
        self.y_test = np.array(test_data['label'].values)
        self.X_test_tkd = None
        vocab_size = 87912
        self.tokenizer = Tokenizer(vocab_size, oov_token='OOV')
        self.tokenizer.fit_on_texts(X_train)
        self.max_len = 250

    def preprocessing(self):
        okt = Okt()
        R_frm = self.df.copy()
        R_frm['리뷰'] = R_frm['리뷰'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)
        R_frm['리뷰'].replace('', np.nan, inplace=True)
        R_frm['리뷰'] = R_frm['리뷰'].astype(str)
        stopwords = ['도', '는', '다', '의', '가', '이', '은',
                     '한', '에', '하', '고', '을', '를', '인', '듯',
                     '과', '와', '네', '들', '듯', '지', '임', '게',
                     '는', '이', '했', '슴', '음', '것', '거', '로',
                     '들', '거', '곳', '분', '원', '더', '왜', '해',
                     '수', '할', '그', '함', '돈', '번', '두', '개',
                     '건', '내', '저', '만', '갈', '걸', '제', '명',
                     '분',
                     '인데', '이가', '했었', '해서',
                     '습니다', '했는데', '입니다']

        okt = Okt()
        R_frm['tokenized'] = R_frm['리뷰'].apply(okt.pos)
        R_frm['tokenized'] = R_frm['tokenized'].apply(
            lambda x: [word for word, shape in x if shape in ['Verb', 'Adjective', 'Noun', 'VerbPrefix'] if word not in stopwords])
        R_pred = R_frm['tokenized'].values
        return R_pred

    def model_test(self):
        tokenizer = self.tokenizer
        X_test = tokenizer.texts_to_sequences(self.X_test)
        y_test = self.y_test
        X_test = pad_sequences(X_test, maxlen=self.max_len)
        print("\n 테스트 정확도: %.4f" % (self.loaded_model.evaluate(X_test, y_test)[1]))

    def word_index(self):
        tokenizer = self.tokenizer
        print(tokenizer.word_index)

    def predict(self):
        R_pred = self.preprocessing()
        max_len = self.max_len
        tokenizer = self.tokenizer
        result = []
        pred = tokenizer.texts_to_sequences(R_pred)
        pred = pad_sequences(pred, maxlen=max_len)
        score = self.loaded_model.predict(pred)
        return score

    def prediction(self):
        result = []
        score = self.predict()
        for num in range(len(score)):
            if score[num][0] > 0.9:
                result.append('0')
            elif score[num][0] < 0.1:
                result.append('1')
            else:
                result.append('2')
        return result


In [9]:
import pandas as pd
data = pd.read_excel('model_test_data.xlsx')
# data

In [12]:
test = psng_predict_all(data)

2023-06-04 14:56:04.813637: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-04 14:56:04.815345: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-04 14:56:04.816585: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [13]:
test.prediction()

JVMNotFoundException: No JVM shared library file (libjvm.so) found. Try setting up the JAVA_HOME environment variable properly.